In [1]:
import sqlite3
import pandas as pd
from pathlib import Path


def list_tables(db_path: str):
    """
    Returns a list of all table names in the SQLite .db database.
    """
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        # Debug: show raw results
        print("Raw table query result:", tables)
        
        return [t[0] for t in tables]
    
    except sqlite3.Error as e:
        print("Database error:", e)
        return []
    
    finally:
        if 'conn' in locals():
            conn.close()


def load_table(db_path: str, table_name: str) -> pd.DataFrame:
    """
    Loads a specific table from a SQLite .db file into a pandas DataFrame.
    """
    try:
        conn = sqlite3.connect(db_path)
        df = pd.read_sql(f"SELECT * FROM {table_name};", conn)
        return df
    
    except sqlite3.Error as e:
        print(f"Error loading table '{table_name}':", e)
        return pd.DataFrame()
    
    finally:
        if 'conn' in locals():
            conn.close()


if __name__ == "__main__":
    # ---- CONFIG ----
    db_path = "mental_health_assessments.db"    # <-- Change this
    table_to_load = None            # Optional: set a table name
    
    # Check file exists
    if not Path(db_path).exists():
        print(f"❌ Error: Database file not found: {db_path}")
        exit()
    
    # ---- LIST TABLES ----
    print("\n🔍 Listing tables in database...")
    tables = list_tables(db_path)
    
    if not tables:
        print("\n❌ No tables found in this database.")
        print("➡️ This usually means:")
        print("   - The DB is empty")
        print("   - Wrong DB file")
        print("   - Database was created but no tables inserted")
        exit()
    
    print("\n📌 Tables found:", tables)

    # If user did not pick a table, load the first one
    if table_to_load is None:
        table_to_load = tables[0]
        print(f"\nℹ️ No table specified. Loading first table: {table_to_load}")
    
    # ---- LOAD TABLE ----
    print(f"\n📥 Loading table '{table_to_load}'...")
    df = load_table(db_path, table_to_load)
    
    if df.empty:
        print(f"\n⚠️ Table '{table_to_load}' is empty or failed to load.")
    else:
        print("\n🟦 Data Preview:")
        print(df.head())

    print("\n✅ Done.")


🔍 Listing tables in database...
Raw table query result: [('assessments',)]

📌 Tables found: ['assessments']

ℹ️ No table specified. Loading first table: assessments

📥 Loading table 'assessments'...

🟦 Data Preview:
                 id              assessment_timestamp  \
0  MH20251124233131  2025-11-24T23:31:31.569000-05:00   
1  MH20251124233825  2025-11-24T23:38:25.688000-05:00   

                   report_timestamp          timezone patient_name  \
0  2025-11-24T23:31:31.620250-05:00  America/New_York          tom   
1  2025-11-24T23:38:25.701063-05:00  America/New_York        tomee   

  patient_number patient_age patient_gender primary_diagnosis  confidence  \
0             t1          33           Male    Bipolar Type-1   74.685207   
1             t2          32         Female    Bipolar Type-2   94.024335   

   confidence_percentage                                 all_diagnoses_json  \
0              74.685207  [{"confidence_percentage": 74.6852070561985, "...   
1         

In [2]:
df

,id,assessment_timestamp,report_timestamp,timezone,patient_name,patient_number,patient_age,patient_gender,primary_diagnosis,confidence,confidence_percentage,all_diagnoses_json,responses_json,processing_details_json,technical_details_json,clinical_insights_json,created_at
0,MH20251124233131,2025-11-24T23:31:31.569000-05:00,2025-11-24T23:31:31.620250-05:00,America/New_York,tom,t1,33,Male,Bipolar Type-1,74.685207,74.685207,"[{""confidence_percentage"": 74.6852070561985, ""...","{""Mood Swing"": ""YES"", ""Sadness"": ""Sometimes"", ...","{""assessment_start_time"": ""2025-11-24T23:31:31...","{""composite_scores_included"": true, ""feature_a...","{""adjustment_reasons"": [], ""confidence_adjustm...",2025-11-25 04:31:31
1,MH20251124233825,2025-11-24T23:38:25.688000-05:00,2025-11-24T23:38:25.701063-05:00,America/New_York,tomee,t2,32,Female,Bipolar Type-2,94.024335,94.024335,"[{""confidence_percentage"": 94.02433518507401, ...","{""Mood Swing"": ""YES"", ""Sadness"": ""Sometimes"", ...","{""assessment_start_time"": ""2025-11-24T23:38:25...","{""composite_scores_included"": true, ""feature_a...","{""adjustment_reasons"": [], ""confidence_adjustm...",2025-11-25 04:38:25
